In [95]:
import pandas as pd
import numpy as np

In [96]:
df = pd.read_json("allwords.json")

In [97]:
langs = ['en', 'es']

In [128]:
en_pronouns = {'I': 1, 'you': 2, 'he/she': 3, 'we': 4, 'they': 6}
es_pronouns = {'yo': 1, 'tú': 2, 'él/ella/Ud.': 3, 'Ud.': 3, 'nosotros': 4, 'vos': 5, 'vosotros': 5, 'ellos/ellas/Uds.': 6, 'Uds.': 6}

In [99]:
es_paradigms = set(['infinitive'])
for words in df['translations']:
    for translation in words:
        if 'spanConj' in translation:
            for conj in translation['spanConj']:
                es_paradigms.add(conj['paradigm'])
                
en_paradigms = set(['infinitive'])
for words in df['engConj']:
    for conj in words:
        en_paradigms.add(conj['paradigm'])

In [100]:
data = pd.DataFrame(columns=pd.MultiIndex.from_product([langs, es_paradigms, list(en_pronouns.values())]))

In [144]:
all_rows = []

for ind, row in df.iterrows():
    data_row = {}
    en_conj = row['engConj']
    for conj in en_conj:
        data_row[('en', conj['paradigm'], en_pronouns[conj['pronoun']])] = conj['word']
    for i in [1,2,3,4,6]:
        data_row[('en', 'infinitive', i)] = row['engWord']
        
    for translation in row['translations']:
        if 'spanConj' not in translation:
            continue
        
        enes_row = data_row.copy()
        for conj in translation['spanConj']:
            enes_row[('es', conj['paradigm'], es_pronouns[conj['pronoun']])] = conj['word']
        for i in [1,2,3,4,6]:
            enes_row[('es', 'infinitive', i)] = translation['meaning']
            
        all_rows.append(enes_row)

In [146]:
data = data.append(all_rows, ignore_index=True)

In [147]:
data.to_csv("220417_enes_conjugations.csv", index=False)